In [1]:
import pandas as pd

# coding: utf-8

# 만든 기능 임포트
from crawling import UrlCrawling, ContentCrawling
from preprocessing import Preprocessing
from clustering import Clustering
from summary import Summary
from database import insert, select
from tts import tts

In [2]:
# url 크롤링
url_crawler = UrlCrawling()

six_url_df = url_crawler.getSixUrl()                          # 6개 카테고리 (카테고리, url 데이터프레임)
entertainment_url_df = url_crawler.getEntertainmentUrl()      # 연예 (카테고리, url 데이터프레임)
sports_url_df = url_crawler.getSportsUrl()                    # 스포츠 (카테고리, url 데이터프레임)

six_url_df, entertainment_url_df, sports_url_df = url_crawler.removeDuplicationUrl(six_url_df, entertainment_url_df, sports_url_df)   # 이미 요약한 기사 제거

# 본문 크롤링
six_url, entertainment_url, sports_url = list(six_url_df['six_url']), list(entertainment_url_df['entertainment_url']), list(sports_url_df['sports_url'])
all_url = six_url + entertainment_url + sports_url
category = list(six_url_df['category']) + list(entertainment_url_df['category']) + list(sports_url_df['category'])

content_crawler = ContentCrawling([], [], [], [], [])

content_crawler.getSixContent(six_url)
content_crawler.getEntertainmentContent(entertainment_url)
content_crawler.getSportsContent(sports_url)

article_df = content_crawler.makeDataFrame(all_url, category)                   # 본문 데이터프레임 생성

article_df = Preprocessing.getNouns(article_df)                                 # 명사 추출

Preprocessing.removeEnglishArticle(article_df)                                  # 영어 기사 삭제

vector_list = Preprocessing.getVector(article_df)                               # 명사 벡터화

Clustering.addClusterNumber(article_df, vector_list)                            # 군집 번호 열 생성
cluster_counts_df = Clustering.getClusteredArticle(article_df)                  # 군집 개수 카운트한 df

summary_article = Summary.getSummaryArticle(article_df, cluster_counts_df)              # 요약한 기사 데이터 프레임 반환
summary_article = Preprocessing.convertCategory(summary_article)

정치 1 페이지
경제 1 페이지
사회 1 페이지
생활/문화 1 페이지
세계 1 페이지
IT/과학 1 페이지
연예 1 페이지
스포츠 1 페이지
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15 요약봇 X, 16, 17, 18, 19 요약봇 X, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91 요약봇 X, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 
1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 

In [ ]:
del summary_article['naver_summary']
del summary_article['similarity']

insert(summary_article.values.tolist())

In [ ]:
# 필요할 때 쓰는 코드
# csv로 저장
summary_article.to_csv("csv/유사도측정.csv",index=False, encoding="utf-8-sig")
df = pd.read_csv("csv/유사도측정.csv")
tts(df)
insert(summary_article.values.tolist())
select()

In [12]:
from keybert import KeyBERT
from konlpy.tag import Okt

key_model = KeyBERT('paraphrase-multilingual-MiniLM-L12-v2')  #distilbert-base-nli-mean-tokens / paraphrase-multilingual-MiniLM-L12-v2

for i in range(len(summary_article)):
    title = " ".join(summary_article['nouns'][i])

    result = key_model.extract_keywords(title, keyphrase_ngram_range=(2, 3), top_n=1, use_mmr=True)
    print(result)


def test_nouns(article_df):
    okt = Okt()
    nouns_list = []                               # 명사 리스트

    for content in article_df["content"]:
        nouns_list.append(okt.phrases(content))     # 명사 추출 (리스트 반환)

    article_df["nouns"] = nouns_list              # 데이터 프레임에 추가

    return article_df

[('범죄 처벌 지상', 0.6613)]
[('나라 비대 위원장', 0.6807)]
[('주장 대표 측은', 0.6406)]
[('윤석열 대통령 여당', 0.8292)]
[('이재명 대표 민주당', 0.8013)]
[('이상 서명 파악', 0.7063)]
[('민주당 겨냥', 0.7366)]
[('압박 민주당 특검', 0.6977)]
[('위원장 민주당', 0.8064)]
[('박근혜 정부 청와대', 0.7932)]
[('연말 배당 효과', 0.6976)]
[('배송 로봇 시대', 0.746)]
[('투자 대표 인도', 0.6746)]
[('주택 허가 올해', 0.6608)]
[('투자자 탁금 이용', 0.7624)]
[('재직 회사 비율', 0.7264)]
[('금리 폭풍 희비', 0.7889)]
[('삼성 한국 정부', 0.7096)]
[('시설 배터리 업계', 0.7727)]
[('외국인 투자자 수익률', 0.8108)]
[('대학 이월 전국', 0.7103)]
[('청장 지난해 지방선거', 0.7734)]
[('범죄 국민 충격', 0.7058)]
[('산업 중심 구미', 0.5634)]
[('개통 영덕 서울', 0.6742)]
[('거부권 행사 반대', 0.7014)]
[('뉴스타파 김만 배신', 0.6389)]
[('경찰서 교수 인권', 0.8026)]
[('재판 곧바로 가스', 0.7049)]
[('성탄절 서울 도봉구', 0.699)]
[('아이 시야 갑자기', 0.8137)]
[('사진기 당황 주저', 0.6009)]
[('새벽 이불 노크', 0.7427)]
[('스포츠 패키지 가솔린', 0.6838)]
[('화장실 제이콥 출산', 0.7099)]
[('맥주 시장 선도', 0.7136)]
[('성현 롯데 마트', 0.7933)]
[('췌장 당뇨병 협심증', 0.671)]
[('지표조사 신증동국여지승람 문씨', 0.8788)]
[('새해 보급 전기차', 0.8086)]
[('공격 미군 중태', 0.7737)]
[('바이든 대통령 대결', 0.6834